## Conv 연산
- [첫 예제](./deep-learning-with-python/5.1-introduction-to-convnets.ipynb)
- FC layer(Dense)는 입력 특성 곤간에 있는 전역 패턴을 학습
- Conv layer는 **로컬 패턴(지역 패턴)**을 학습
- 흥미로운 성질
    - 학습된 패턴은 **평행 이동 불변성(translation invariant)**를 가진다
        - 특정 위치에서 학습된 패턴은 다른 위치에서도 인식 가능
    - 컨브넷은 패턴의 **공간적 계층 구조**를 학습 할 수 있음
        - 첫 번째 합성곱 층에서 **에지** 같은 작은 지역 패턴을 학습
            - 이미지는 기본적으로 **Edge, Texture** 등의 로컬 패턴으로 분해된다
        - 두 번째 합성곱 층은 첫 번째 층의 특성으로 구성된 패턴 학습
        - 이런식으로 매우 복잡하고 추상적인 시각적 개념을 학습
- Conv 연산은 **Feature Map이라고 부르는 3D 텐서(높이, 너비, 깊이)**에 적용된다
- Conv 연산은 **입력 특성 맵**에서 작은 **Patch**들을 추출하고 이런 모든 패치에 같은 변환을 적용해 **Output Feature Map**을 만든다
- 출력 특성 맵도 3D 텐서
    - 깊이는 컨브 레이어의 매개변수로 결정됨 (filter 갯수)
        - 필터는 입력 데이터의 어떤 특성을 인코딩 (ex, 얼굴이 있는지 없는지)
    - filter 하나의 크기는 (patch_height, patch_width, **input_depth**)
        - 기본적으로 한 필터는 모든 채널을 동시에 고려
        - 이전 예시의 첫 컨브 레이어는 (3, 3, 1) 필터를 32개 적용, 두 번째 컨브레이어는 (3, 3, 32) 크기의 필터를 64개 적용 ...
        - 필터 하나에 대한 결과는 **Response Map**이라고 한다 
            - 입력의 각 위치에서 필터의 패턴에 대한 **응답**(존재 유무를 정량화)을 나타내기 때문
- 핵심 파라미터
    - 입력으로부터 뽑아낼 **패치의 크기**(필터)
    - **필터의 수**
- 합성곱 커널?
    - 합성곱 층의 필터를 하나의 행렬로 합친 것
    - ex) (3, 3, 1, 32), (3, 3, 32, 64)
- 입력과 출력의 크기(높이 너비)는 서로 다를 수 있다.
    - 패딩(**padding**)을 추가함으로써 같게 유지 가능(zero padding)
        - 입력 피쳐 맵의 모든 위치[i,j]에 필터의 중앙을 위치 시킬 수 없다보니 출력의 크기가 줄어들기 마련
        - padding=valid 기본 값, **padding=same** 입력과 동일한 높이 및 너비를 가지는 출력을 만들게끔 패딩
            - 5x5 인풋에 3x3 필터라면 5x5 => 7x7로 만들어주면 됨
    - 스트라이드(**stride**) 사용 여부에 따라 달라짐
        - 기본값 1 (위 아래로 한칸씩 이동)
        - 2로 바꿀 경우 특성 맵의 너비와 높이가 2의 배수로 다운샘플링(경계 문제가 있을 경우 더 작아짐)
        - 드물게 사용되며, 다운샘플링을 위해서는 **Max Pooling** 연산이 더 자주 쓰임

## 최대 풀링 연산(Max Pooling)
- Down Sampling
- 입력 특성 맵에서 윈도우에 맞는 패치 추출하고, 각 채널별로 최댓값을 출력
- 보통 2x2 window 2 stride를 적용해서 특성 맵을 절반 크기로 다운샘플링
    - 합성곱은 전형적으로 3x3 window stride 1
- 특성맵을 **다운샘플링 하는 이유?**
    - 합성곱으로만 네트워크를 구성하면, 특성의 공간적 계층 구조를 학습하는 데 도움이 안됨
        - 만약 3x3 filter를 3개의 레이어에 걸쳐 적용했다면, 3번째 층의 3x3 윈도우는 초기 입력의 7x7 윈도우 영역에 대한 정보만 담고 있음
            - 7x7 <-> 5x5 <-> 3x3
        - 7x7 픽셀만 보고 분류작업을 해야하는 셈
        - 따라서, 더 나은 분류를 위해 마지막 합성곱 층의 특성이 전체 입력에 대한 정보를 가지고 있어야 한다.
    - 나아가, 가중치가 너무 많아지기 때문에 다운샘플링이 필요
    - 정리하면
        - **가중치 개수를 줄이고**
        - 연속된 합성곱 층이 점점 커진 윈도우를 통해 바라볼 수 있도록 만들어 필터의 **공간적 계층 구조 형성**

## Pooling
- 최대 풀링 말고 평균 풀링도 있긴 함
    - 최대 풀링이 더 잘 동작하는 경향
        - 특성 맵의 각 타일에서 어떤 패턴이나 개념의 **존재 여부**를 인코딩 하는 경향이 있기 때문
    - 가장 납득할만한 서브샘플링 전략
        1. 합성곱으로 조밀한 특성맵 만들기
        2. 작은 패치에 대해 최대로 활성화된 특성 고르기
    - 평균 전략을 취하다 괜히 특성 정보를 희석시킬 수가 있다.

## 이후 과정
- [기본 예제 및 데이터 증식](./deep-learning-with-python/5.2-using-convnets-with-small-datasets.ipynb)
- [pre trained model 사용](./deep-learning-with-python/5.3-using-a-pretrained-convnet.ipynb)
- [시각화](./deep-learning-with-python/5.4-visualizing-what-convnets-learn.ipynb)

## 흥미로운 자료
- https://github.com/songjein/object-localization
- https://github.com/songjein/YOLOw-Keras